In [2]:

%pip install tf-keras

import numpy as np 
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------ --------------------------- 0.5/1.7 MB 670.4 kB/s eta 0:00:02
   ------------ --------------------------- 0.5/1.7 MB 670.4 kB/s eta 0:00:02
   ------------ --------------------------- 0.5/1.7 MB 670.4 kB/s eta 0:00:02
   ------------------ --------------------- 0.8/1.7 MB 524.3 kB/s eta 0:00:02
   ------------------------ --------------- 1.0/1.7 MB 621.2 kB/s eta 0:00:02
   ------------------------ --------------- 1.0/1.7 MB 621.2 kB/s eta 0:00:02
   ------------------------------ --------- 1.3/1.7 MB 670.9 kB/s eta 0:00:01
   ------------------------------ --------- 1.3/1.7 MB 670.9 kB/s eta 0:00:01
   ----------------------------

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\tejas\\AppData\\Local\\Temp\\pip-unpack-c7mag8_v\\tensorflow-2.20.0-cp311-cp311-win_amd64.whl'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


ValueError: Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.